# Analisi dati per l'esperienza di Michelson

in realtà l'analisi dati si potrebbe fare anche in excel ma è brutto come la francia quindi no.

### Import librerie

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import scienceplots

plt.style.use(["science", "grid", "ieee"])